# Trabajo Práctico NLP: Detección de Tópicos y Clasificación

**Mariela Iaccarino**

Certificación Experta en NLP - ITBA

Con este código se realiza la detección de tópicos en textos  entrenando modelos de tópicos diariamente con BERTopic, y luego fusionando estos modelos en un único modelo combinado. Los tópicos resultantes se almacenan en OpenSearch, lo que permite realizar búsquedas y clasificaciones eficientes de nuevos documentos, incluyendo la extracción de entidades y análisis de sentimiento. El proceso facilita el análisis y agrupamiento de grandes volúmenes de texto a lo largo del tiempo.

Las diferencias clave entre el código anterior "TP_NLP_2_ProcesamientoDiario" radican en el enfoque del procesamiento diario de tópicos, la comparación entre días, la generación de nuevos tópicos, y el almacenamiento en la base de datos.

Aquí están las principales diferencias:

**Flujo de Procesamiento:**

*TP_NLP_2_ProcesamientoDiario:* Tiene un enfoque iterativo y dinámico. Primero, entrena el modelo de tópicos solo con los datos del primer día, guarda estos tópicos en la base de datos, y luego, para cada día siguiente, compara los nuevos tópicos con los existentes para decidir si debe crear nuevos tópicos o fusionarlos con los existentes.

*TP_NLP_3_MergeModels:* Se enfoca en entrenar modelos de tópicos diariamente y luego fusionar esos modelos en un solo modelo combinado. Después de la fusión, se almacenan los embeddings de los tópicos en OpenSearch.

**Comparación entre Días:**

*TP_NLP_2_ProcesamientoDiario:* Compara los tópicos de cada día con los tópicos existentes en la base de datos y decide si fusionarlos o crear un nuevo tópico. Esto evita la proliferación de tópicos redundantes.

*TP_NLP_3_MergeModels:* No compara los tópicos generados en diferentes días antes de fusionarlos; simplemente los fusiona al final del procesamiento diario.

**Almacenamiento y Fusión de Tópicos:**

*TP_NLP_2_ProcesamientoDiario:* Los tópicos se almacenan inmediatamente después de ser procesados y, si un nuevo tópico es similar a uno existente, se fusionan antes de almacenarse. La fusión se realiza sobre la marcha en lugar de al final del procesamiento diario.

*TP_NLP_3_MergeModels:* Los tópicos de cada día se fusionan en un modelo combinado final, y este modelo fusionado es el que se almacena en OpenSearch.

**Generación de Nuevos Tópicos:**

*TP_NLP_2_ProcesamientoDiario:* Decide si un nuevo tópico debe ser creado o fusionado con uno existente antes de almacenarlo, lo que permite una evaluación más dinámica y precisa de los tópicos día a día.

*TP_NLP_3_MergeModels:* Genera nuevos tópicos basados en la probabilidad y la similitud entre los documentos al final del proceso.

## 1. Configuración y Carga de Datos
1.1 Importación de librerías necesarias:

In [2]:
!pip install datasets umap-learn chromadb hdbscan sentence_transformers BERTopic matplotlib opensearch-py==2.3.0


In [3]:
from datasets import load_dataset
import pandas as pd
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from datetime import datetime
from transformers import pipeline

from sklearn.metrics.pairwise import cosine_similarity
from opensearch_data_model import Topic, TopicKeyword, os_client
from datetime import datetime
from dateutil.parser import parse
from utils import SPANISH_STOPWORDS
import numpy as np
import matplotlib.pyplot as plt

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1.2 Funciones

In [4]:
def topic_threshold(topic_id, topic_model, probs):
    try:
        docs_per_topics =  [i for i, x in enumerate(topic_model.topics_) if x== topic_id    ]
        return np.array(  [probs[doc_idx] for doc_idx in docs_per_topics  ]).mean() 
    except:
        return 0


In [5]:
def delete_index_opensearch(index_name: str) -> bool:

    try:
        # Consulta para eliminar todos los documentos
        delete_query = {
                        "query": {
                        "match_all": {}
                        }
        }

        # Ejecutar la operación de borrado por consulta
        response = os_client.delete_by_query(index=index_name, body=delete_query)

        return True

    except Exception as e:
        print(f"Ha ocurrido un error: {e}")
        return

In [6]:


def topic_threshold(topic_id, merged_model_final, probs):
    try:
        # Obtener los documentos asociados con el topic_id en el modelo fusionado
        docs_per_topic = [i for i, x in enumerate(merged_model_final.topics_) if x == topic_id]
        
        # Calcular el umbral de probabilidad como el promedio de probabilidades para los documentos del tópico
        return np.array([probs[doc_idx] for doc_idx in docs_per_topic]).mean()
    except:
        return 0

In [7]:
def get_topic_name(keywords):
    return ', '.join([k for k, s in keywords[:4]])

In [8]:

# Configuración de pipelines para NER y análisis de sentimiento
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Función para extracción de entidades
def extract_entities(text):
    entities = ner_pipeline(text)
    return [entity['word'] for entity in entities]

# Función para análisis de sentimiento
def analyze_sentiment(text):
    sentiment = sentiment_pipeline(text)
    return sentiment[0]


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
Topic.init()

1.3 Carga y preparación de datos:

Se carga un conjunto de datos de noticias de 5 días y se le agrega un campo de fecha.

In [10]:

# Función para agregar un campo fecha a un dataset
def add_fecha_field(dataset, fecha):
    df = pd.DataFrame(dataset['train'])
    df['date'] = fecha
    return df

# Cargar datasets y agregar el campo fecha
datasets = [
    ("jganzabalseenka/news_2024-07-01_24hs", '2024-07-01'),
    ("jganzabalseenka/news_2024-07-12_24hs", '2024-07-12'),
    ("jganzabalseenka/news_2024-07-14_24hs", '2024-07-14'),
    ("jganzabalseenka/news_2024-07-16_24hs", '2024-07-16'),
    ("jganzabalseenka/news_2024-07-19_24hs", '2024-07-19')
    
]


df_list = [add_fecha_field(load_dataset(ds[0]), ds[1]) for ds in datasets]
df = pd.concat(df_list, ignore_index=True)

# Seleccionar una muestra de 1000 textos por día
df_sample = df.groupby('date').apply(lambda x: x.sample(min(len(x), 10000))).reset_index(drop=True)



In [11]:
# Mostrar una muestra del dataset
df[['title', 'text', 'date']].head()


,title,text,date
0,"Domingo, 30 de junio de 2024 (24:00 GMT)","30 Jun, 2024 Por Newsroom Infobae Nuevo FRANC...",2024-07-01
1,"Encontraron un golpe, dos pelos y manchas roja...","1 Jul, 2024 Por Federico Fahsbender Nuevo En ...",2024-07-01
2,Las cinco enfermedades por las que se ha conce...,"1 Jul, 2024 Por Diego Mariño Nuevo La pensión...",2024-07-01
3,"Así fue el espectacular cumpleaños de Vida, la...","30 Jun, 2024 Nuevo La creatividad fluye por la...",2024-07-01
4,Clima en Madrid: conoce el pronóstico y prepár...,"1 Jul, 2024 Por Infobae Noticias Nuevo Los pr...",2024-07-01


1.4 Configuración del Modelo de Embeddings y Otros Modelos:

In [12]:

# Configuración del modelo de embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configuración de UMAP para reducción de dimensionalidad
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Configuración de HDBSCAN para clustering
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)



## 2. Procesamiento y Detección de Tópicos Diarios

Entrenamiento Diario: 

Para cada día de datos, se entrena un modelo de detección de tópicos utilizando BERTopic. Este proceso implica generar embeddings para los textos y luego aplicar UMAP para reducir la dimensionalidad y HDBSCAN para agrupar los textos en tópicos.

In [13]:



# Entrenar el modelo de tópicos para cada día
def train_topic_model_for_day(df_day):
    texts = df_day['text'].tolist()
    entities = set(sum(list([list(e) for e in df_day['entities_transformers'].values]), []))
    keywords = set(sum(list([list(e) for e in df_day['keywords'].values]), []))
    all_tokens = list(entities.union(keywords))
# Configurar el vectorizador
    tf_vectorizer = CountVectorizer(
        max_df=0.9,
        min_df=0.1,
        ngram_range=(1, 3),
        stop_words=SPANISH_STOPWORDS,
        lowercase=False,
        vocabulary=all_tokens,)
    
    topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=tf_vectorizer,
    ctfidf_model=ClassTfidfTransformer(),
    representation_model=KeyBERTInspired(),
    language='spanish'
)
    topics, probs = topic_model.fit_transform(texts)
    df_day['topic'] = topics
    df_day['probs'] = probs
    return df_day, topic_model

# Separar el dataframe por día y entrenar el modelo
df_day_1 = df[df['date'] == '2024-07-01']
df_day_2 = df[df['date'] == '2024-07-12']
df_day_3 = df[df['date'] == '2024-07-14']
df_day_4 = df[df['date'] == '2024-07-16']
df_day_5 = df[df['date'] == '2024-07-19']

df_day_1, topic_model_1 = train_topic_model_for_day(df_day_1)
df_day_2, topic_model_2 = train_topic_model_for_day(df_day_2)
df_day_3, topic_model_3 = train_topic_model_for_day(df_day_3)
df_day_4, topic_model_4 = train_topic_model_for_day(df_day_4)
df_day_5, topic_model_5 = train_topic_model_for_day(df_day_5)



/home/RECONQUISTA/a118672/.local/lib/python3.8/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df) + 1)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable thi

In [14]:
df_day_1[['title', 'topic', 'probs']].head()

,title,topic,probs
0,"Domingo, 30 de junio de 2024 (24:00 GMT)",5,1.000000
1,"Encontraron un golpe, dos pelos y manchas roja...",115,0.759449
2,Las cinco enfermedades por las que se ha conce...,-1,0.000000
3,"Así fue el espectacular cumpleaños de Vida, la...",-1,0.000000
4,Clima en Madrid: conoce el pronóstico y prepár...,-1,0.000000


In [15]:
df_day_2[['title', 'topic', 'probs']].head()

,title,topic,probs
16025,De las quemaduras al melanoma: estos son los p...,25,1.000000
16026,Guillermo Varela recibió la roja en el pospart...,21,1.000000
16027,Martín de la Puente opta a ser primer español ...,18,0.412598
16028,El bonito pueblo de Asturias que fue la primer...,-1,0.000000
16029,China usaría a Perú como su 'chacra': financia...,115,1.000000


In [16]:
df_day_3[['title', 'topic', 'probs']].head()

,title,topic,probs
33868,"Andy Samberg reveló que ""Saturday Night Live"" ...",-1,0.000000
33869,A qué hora juegan Argentina vs Colombia HOY en...,2,0.943393
33870,Organismo de transparencia de México indaga ha...,55,0.561608
33871,Josepmir Ballón aconsejó a Paolo Guerrero tras...,-1,0.000000
33872,La líder del opositor Partido de los Trabajado...,-1,0.000000


In [17]:
df_day_4[['title', 'topic', 'probs']].head()

,title,topic,probs
41053,ATU responde al Metropolitano y afirma descono...,102,0.073565
41054,"Kevin Serna, preparado para sumarse a Fluminen...",-1,0.000000
41055,La acusación popular solicita al juez Peinado ...,6,1.000000
41056,La ola de detenciones de opositores marca la c...,6,0.869119
41057,Incautaron 200 piezas de armas de grueso calib...,-1,0.000000


In [18]:
df_day_5[['title', 'topic', 'probs']].head()

,title,topic,probs
58336,"El día de las ""pantallas azules de la muerte"":...",1,1.000000
58337,Boluarte seguirá siendo investigada por 'Rolex...,42,0.177063
58338,"""Esto es una venganza, hay una interna familia...",2,0.785909
58339,Escándalo en Ecuador: Independiente del Valle ...,-1,0.000000
58340,El tango con el que Cacho Castaña rebautizó a ...,-1,0.000000


### 3. Fusión de Modelos 

Fusión de Modelos: Después de entrenar los modelos diarios, se fusionan para crear un modelo combinado que pueda identificar y agrupar tópicos similares a lo largo de múltiples días.


In [19]:

# Fusionar los modelos
merged_model = BERTopic.merge_models([topic_model_1, topic_model_2])
merged_model_2 = BERTopic.merge_models([merged_model, topic_model_3])
merged_model_3 = BERTopic.merge_models([merged_model_2, topic_model_4])
merged_model_final = BERTopic.merge_models([merged_model_3, topic_model_5])


In [20]:
len(merged_model_final.get_topic_info())

179

In [21]:
merged_model_final.get_topic_info().tail(5)

,Topic,Count,Name,Representation,Representative_Docs
174,173,16,173_jubilados pensionados_decreto publicado_me...,"[jubilados pensionados, decreto publicado, men...",NaN
175,174,103,174_coordinación_Los Libertadores_habilitación...,"[coordinación, Los Libertadores, habilitación,...",NaN
176,175,52,175_Shannen Doherty_Doherty_Leslie Sloane_Sloane,"[Shannen Doherty, Doherty, Leslie Sloane, Sloa...",NaN
177,176,17,176_privacidad_Interior_La Voz_consultas,"[privacidad, Interior, La Voz, consultas, Hugo...",NaN
178,177,39,177_julio_2020_Nacionales_Rosario Finanzas,"[julio, 2020, Nacionales, Rosario Finanzas, de...",NaN


### 4. Almacenamiento en OpenSearch

Almacenamos los embeddings de los tópicos en OpenSearch y verificamos el almacenamiento:

In [22]:
delete_index_opensearch("topic")

True

In [23]:

# Concatenar los DataFrames
df_combined = pd.concat([df_day_1, df_day_2,df_day_3, df_day_4,df_day_5])

# Obtener la fecha mínima y máxima
min_date = df_combined['date'].min()
max_date = df_combined['date'].max()

# Mostrar resultados
print(f"Fecha mínima: {min_date}")
print(f"Fecha máxima: {max_date}")

Fecha mínima: 2024-07-01
Fecha máxima: 2024-07-19


In [24]:
texts = df_combined['text'].tolist()

In [25]:
docs, probs = merged_model_final.transform(texts)

In [26]:

# Obtener embeddings de los tópicos

# Calcular la similitud de coseno entre los tópicos del modelo fusionado
embedings = merged_model_final.embedding_model.embed(texts)
sim_matrix = cosine_similarity(
    merged_model_final.topic_embeddings_,
    embedings
)

Almacenamiento en OpenSearch
Guardar Tópicos: Los embeddings de los tópicos detectados y fusionados se almacenan en OpenSearch. Esto permite realizar búsquedas rápidas y eficientes sobre los tópicos utilizando la búsqueda vectorial.

In [27]:
for topic in merged_model_final.get_topics().keys():
    if topic > -1:
        print(topic)
        keywords = merged_model_final.topic_representations_[topic]
        topic_keywords = [TopicKeyword(name=k, score=s) for k, s in keywords]
        threshold = topic_threshold(topic, merged_model_final, probs)

        # Check to ensure topic + 1 is within bounds of sim_matrix
        if topic + 1 < len(sim_matrix):
            best_doc_index = sim_matrix[topic + 1].argmax()
            best_doc = df_combined.iloc[best_doc_index].text

            topic_doc = Topic(
                vector=list(merged_model_final.topic_embeddings_[topic + 1]),
                similarity_threshold=threshold,
                created_at=datetime.now(),
                to_date=parse(max_date),
                from_date=parse(min_date),
                index=topic,
                keywords=topic_keywords,
                name=get_topic_name(keywords),
                best_doc=best_doc
            )

            print(topic_doc.save())
        else:
            print(f"Index {topic + 1} out of bounds for sim_matrix with size {len(sim_matrix)}")


0
created
1
created
2
created
3
created
4
created
5
created
6
created
7
created
8
created
9
created
10
created
11
created
12
created
13
created
14
created
15
created
16
created
17
created
18
created
19
created
20
created
21
created
22
created
23
created
24
created
25
created
26
created
27
created
28
created
29
created
30
created
31
created
32
created
33
created
34
created
35
created
36
created
37
created
38
created
39
created
40
created
41
created
42
created
43
created
44
created
45
created
46
created
47
created
48
created
49
created
50
created
51
created
52
created
53
created
54
created
55
created
56
created
57
created
58
created
59
created
60
created
61
created
62
created
63
created
64
created
65
created
66
created
67
created
68
created
69
created
70
created
71
created
72
created
73
created
74
created
75
created
76
created
77
created
78
created
79
created
80
created
81
created
82
created
83
created
84
created
85
created
86
created
87
created
88
created
89
created
90
created
91
create

### 5. Clasificación de Nuevos Documentos

Clasificación de Textos Nuevos: Para clasificar un nuevo texto, se generan embeddings del texto y se compara con los embeddings almacenados en OpenSearch para encontrar el tópico más similar. Además, se extraen entidades y se realiza un análisis de sentimiento.

In [28]:
Topic.search().count()

178

Inferencia

Finalmente, clasificamos un nuevo texto en los tópicos existentes y extraemos las entidades, keywords y realizamos análisis de sentimiento utilizando pipelines de Hugging Face:

In [ ]:
def classify_text(title, text):
    # Combinar título y texto
    combined_text = title + " " + text
    
    # Generar embeddings del texto combinado
    new_embed = embedding_model.encode([combined_text])  # Asegúrate de usar encode() si es SentenceTransformer
    
    # Construir la consulta KNN para OpenSearch
    query = {
        "size": 5,
        "query": {
            "knn": {
                "vector": {
                    "vector": new_embed[0].tolist(),
                    "k": 1000
                }
            }
        }
    }
    
    # Ejecutar la búsqueda en OpenSearch
    response = os_client.search(index='topic', body=query)
    
    # Verificar si se encontraron resultados
    if response['hits']['hits']:
        # Obtener la información del tópico más cercano
        topic_id = response['hits']['hits'][0]['_id']
        keywords = response['hits']['hits'][0]['_source']['keywords']
        name = response['hits']['hits'][0]['_source']['name']  # Obtener el nombre del tópico
        best_doc = response['hits']['hits'][0]['_source']['best_doc']  # Obtener el mejor documento
        
        # Extraer entidades y análisis de sentimiento
        entities = extract_entities(combined_text)
        sentiment = analyze_sentiment(combined_text)
        
        # Devolver los resultados
        return topic_id, name, keywords, best_doc, entities, sentiment
    
    # Si no se encuentra ningún resultado, devolver None
    return None, None, None, None, None, None


# Ejemplo de clasificación
new_title = "Argentina Campeon"
new_text = "Termino la copa America y Argentina salio campeon"
topic_id, name, keywords, best_doc, entities, sentiment = classify_text(new_title, new_text)
print(f"Tópico: {topic_id}, Nombre: {name}, Keywords: {keywords}, Best Doc: {best_doc}, Entidades: {entities}, Sentimiento: {sentiment}")

In [29]:
for doc in Topic.search().query().scan():
    break

In [30]:
doc.to_dict()

{'vector': [0.009156751446425915,
  0.013354676775634289,
  -0.02184940129518509,
  -0.05864686146378517,
  -0.00856113899499178,
  0.01803724281489849,
  -0.004430643282830715,
  0.04353175684809685,
  0.013458351604640484,
  0.04047631844878197,
  0.032498959451913834,
  -0.034336525946855545,
  -0.0190910492092371,
  0.0017151908250525594,
  0.045677803456783295,
  -0.021010836586356163,
  -0.06220001354813576,
  -0.012244456447660923,
  0.008135088719427586,
  0.030568772926926613,
  0.10981833189725876,
  -0.07996156066656113,
  -0.09240210056304932,
  0.06781023740768433,
  -0.08242151886224747,
  0.004060111008584499,
  -0.017785606905817986,
  0.024220477789640427,
  -0.08877769857645035,
  -0.06715026497840881,
  -0.03803260251879692,
  0.0337878055870533,
  0.06868071109056473,
  0.004016956780105829,
  -0.01765873096883297,
  -0.021284420043230057,
  0.032476674765348434,
  -0.06672663986682892,
  -0.019808493554592133,
  0.051073238253593445,
  -0.06795693933963776,
  -0.00

In [31]:
#from transformers import pipeline

# Configuración de pipelines para NER y análisis de sentimiento
#ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
#sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

def classify_text(title, text, topic_model):
    combined_text = title + " " + text
    new_embed = merged_model_final.embedding_model.embed([combined_text])
    query = {
        "size": 5,
        "query": {
            "knn": {
                "vector": {
                    "vector": new_embed[0].tolist(),
                    "k": 1000
                }
            }
        }
    }
    response = os_client.search(index='topic', body=query)
    if response['hits']['hits']:
        topic_id = response['hits']['hits'][0]['_id']
        keywords = response['hits']['hits'][0]['_source']['keywords']
        entities = extract_entities(combined_text)
        sentiment = analyze_sentiment(combined_text)
        return topic_id, keywords, entities, sentiment
    return None, None, None, None



# Ejemplo de clasificación
new_title = "Argentina Campeon"
new_text = "Termino la copa America y Argentina salio campeon"
topic_id, keywords, entities, sentiment = classify_text(new_title, new_text, merged_model_final)
print(f"Tópico: {topic_id}, Keywords: {keywords}, Entidades: {entities}, Sentimiento: {sentiment}")

Tópico: 0_Ecuador,_Selección_Argentina,_Venezuela,_clasificación, Keywords: [{'name': 'Ecuador', 'score': 0.46930786967277527}, {'name': 'Selección Argentina', 'score': 0.4685828387737274}, {'name': 'Venezuela', 'score': 0.4683346748352051}, {'name': 'clasificación', 'score': 0.4629231095314026}, {'name': 'Primera', 'score': 0.4506341814994812}, {'name': 'Boca', 'score': 0.44130176305770874}, {'name': 'Argentina', 'score': 0.4056209921836853}, {'name': 'México', 'score': 0.3970664441585541}, {'name': 'diferencia', 'score': 0.3854668140411377}, {'name': 'Chile', 'score': 0.36782628297805786}], Entidades: ['Argentina', 'Camp', 'America', 'Argentina'], Sentimiento: {'label': 'POSITIVE', 'score': 0.7363261580467224}


In [32]:
new_doc = "Termino la copa America y Argentina salio campeon"

new_doc_embed = merged_model_final.embedding_model.embed(new_doc)
# %%
new_doc_embed.shape
# %%
query = {
    "size": 5,
    "query": {
        "knn": {
        "vector": {
            "vector": list(new_doc_embed),
            "k" : 1000
        }
        }
    }
}

In [33]:
response = os_client.search(index='topic', body=query)
df_hits = pd.DataFrame(response['hits']['hits'])
winning_topic = Topic.get(df_hits.iloc[0]._id)
winning_topic.to_dict()

{'vector': [0.009156751446425915,
  0.013354676775634289,
  -0.02184940129518509,
  -0.05864686146378517,
  -0.00856113899499178,
  0.01803724281489849,
  -0.004430643282830715,
  0.04353175684809685,
  0.013458351604640484,
  0.04047631844878197,
  0.032498959451913834,
  -0.034336525946855545,
  -0.0190910492092371,
  0.0017151908250525594,
  0.045677803456783295,
  -0.021010836586356163,
  -0.06220001354813576,
  -0.012244456447660923,
  0.008135088719427586,
  0.030568772926926613,
  0.10981833189725876,
  -0.07996156066656113,
  -0.09240210056304932,
  0.06781023740768433,
  -0.08242151886224747,
  0.004060111008584499,
  -0.017785606905817986,
  0.024220477789640427,
  -0.08877769857645035,
  -0.06715026497840881,
  -0.03803260251879692,
  0.0337878055870533,
  0.06868071109056473,
  0.004016956780105829,
  -0.01765873096883297,
  -0.021284420043230057,
  0.032476674765348434,
  -0.06672663986682892,
  -0.019808493554592133,
  0.051073238253593445,
  -0.06795693933963776,
  -0.00